In [53]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

# 環境変数の取得
load_dotenv("../../.env")
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

# PDFドキュメントの読込
documents = SimpleDirectoryReader('./data/pdf').load_data()


from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Settings
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=256,
    chunk_overlap=16,
    tokenizer=tiktoken.encoding_for_model(MODEL_NAME).encode)

# 言語モデルの指定
llm = OpenAI(model=MODEL_NAME, temperature=0.3)

# 設定に反映
Settings.llm = llm
Settings.node_parser = node_parser

# Indexの構築
index = VectorStoreIndex.from_documents(documents)

# ストレージに保存
index.storage_context.persist("./storage01")

from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage01")
# Indexのロード
index = load_index_from_storage(storage_context)

sys_prompt_str = """
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

qa_prompt_str = """
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問: {query_str}
回答："""

refine_prompt_str = """
元の回答を (必要な場合のみ) 以下のコンテキストで改良する機会があります。
-----------
{context_msg}
-----------
新しいコンテキストが与えられた場合、元の回答を改良して、質問 {query_str} に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答: {existing_answer}"""

from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from llama_index.core.agent.workflow import ReActAgent

# テキストQAテンプレートの作成
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# リファインテンプレートの作成
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

agent = ReActAgent(
    llm=llm, 
    similarity_top_k=3,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)

# Chat Engineの作成
#chat_engine = index.as_chat_engine(
#    chat_mode="openai",
#    llm=llm,
#    similarity_top_k=3,
#    text_qa_template=text_qa_template,
#    refine_template=refine_template,
#)
chat_engine = index.as_chat_engine(
    agent=agent,
)

# 質問：1回目
response = chat_engine.stream_chat("公共交通機関の交通費の上限は？")

print(response.sources)
for token in response.response_gen:
    print(token, end="")


# 質問：2回目
response = chat_engine.stream_chat("交通費以外の手当にはどのようなものがありますか？")

for token in response.response_gen:
    print(token, end="")

# 引用元を表示
#for source in response.sources:
    #for source_node in source.raw_output.source_nodes:
        #print("ファイル名：", source_node.metadata["file_name"])
        #print("関連度スコア:", source_node.score)
        #print("テキスト：")
        #print(source_node.node.text)
        #print("-" * 50)  # 区切り線

# 引用元を表示
for source in response.sources:
    print("Raw output:", source.raw_output)  # デバッグ用に追加
    for source_node in source.raw_output:
        # 各要素の内容を表示
        print("Source node:", source_node)

# 引用元を表示
#for source in response.sources:
    #for source_node in source.raw_output:  # リストの各要素にアクセス
for source in response.sources:
    for node_with_score in source.raw_output:
        # NodeWithScore オブジェクトから TextNode を取得
        text_node = node_with_score.node
        # source_node が辞書であると仮定
        print("ファイル名：", text_node.metadata.get("file_name", "不明"))
        print("関連度スコア:", node_with_score.score)
        print("テキスト:", text_node.text)
        print("-" * 50)  # 区切り線



2025-09-01 17:29:57,551 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage01/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage01/index_store.json.


2025-09-01 17:29:58,465 - INFO - Loading all indices.
2025-09-01 17:29:58,469 - INFO - Condensed question: 公共交通機関の交通費の上限は？
2025-09-01 17:29:59,056 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[ToolOutput(blocks=[TextBlock(block_type='text', text="[NodeWithScore(node=TextNode(id_='b259b9fd-0d7a-47b0-9097-1f0bee9991d2', embedding=None, metadata={'page_label': '1', 'file_name': '02賃金規則.pdf', 'file_path': '/Users/hinoue/Downloads/llmdev/00_orientation/14_rag/data/pdf/02賃金規則.pdf', 'file_type': 'application/pdf', 'file_size': 134957, 'creation_date': '2024-11-12', 'last_modified_date': '2024-11-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a16713bb-9f5f-4a86-b1b0-df0753a607e8', node_type='4', metadata={'page_label': '1', 'file_name': '02賃金規則.pdf', 'file_path': '/Users/hinoue/Downloads/llmdev/00_orientation/14_rag/data/pdf/02賃金規則.pdf', 'file_type': 'application/pdf', 'file_size': 134957, 'creat

2025-09-01 17:29:59,806 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


公共交通機関の利用に関する交通費の上限は、月額で3万円まで支給されます。この支給は、実際の経路に基づいて行われ、最安経路をもとに計算されます。

2025-09-01 17:30:01,279 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-01 17:30:01,283 - INFO - Condensed question: 交通費以外にどのような手当がありますか？
2025-09-01 17:30:01,902 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-01 17:30:02,453 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


交通費以外の手当として、以下のものがあります：

1. **住宅手当**: 通勤に1時間以上かかる場合、月額1万円が支給されます。この手当を受けるためには、賃貸契約書など居住地を証明できる書類の提出が必要です。

2. **家族手当**: 扶養家族がいる従業員には家族手当が支給されます。配偶者には月額5,000円、子供1人につき月額3,000円が支給されます（上限は子供3人まで）。

これらの手当は、基本給に加えて支給されるものです。Raw output: [NodeWithScore(node=TextNode(id_='b259b9fd-0d7a-47b0-9097-1f0bee9991d2', embedding=None, metadata={'page_label': '1', 'file_name': '02賃金規則.pdf', 'file_path': '/Users/hinoue/Downloads/llmdev/00_orientation/14_rag/data/pdf/02賃金規則.pdf', 'file_type': 'application/pdf', 'file_size': 134957, 'creation_date': '2024-11-12', 'last_modified_date': '2024-11-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a16713bb-9f5f-4a86-b1b0-df0753a607e8', node_type='4', metadata={'page_label': '1',